# Noneを返さない

In [3]:
x, y = 5, 2

def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs') from e
        
divide(x, y)
divide(0, y)
divide(x, 0)

ValueError: Invalid inputs

# スコープについて

- (0, x)を返していることで(1, x)よりも前にsortされるように渡す値を工夫している(0 < 1)
- クロージャーを使う場合はnonlocalを使用しないと内側の関数が値を更新できない
- nonlocalは使うところは単純な関数のみに限定

In [10]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}

class Sorter(object):
    def __init__(self, group):
        self.group = group
        self.found = False
    
    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)
    
def sort_priority(numbers, group):
    found = False
    def helper(x):
        nonlocal found
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found
    
sorter = Sorter(group)
numbers.sort(key=sorter)
print(numbers)
assert sorter.found is True

found = sort_priority(numbers, group)
print(numbers)
print(found)

[2, 3, 5, 7, 1, 4, 6, 8]
[2, 3, 5, 7, 1, 4, 6, 8]
True


# 大量のデータを扱う場合はジェネレータ式

In [18]:
from itertools import islice

address = 'Four score and seven years ago'

def index_words_iter(text):
    if text:
        yield 0
    for index, letter in enumerate(text):
        if letter == '':
            yield index + 1

def index_file(handle):
    offset = 0
    for line in handle:
        if line:
            yield offset
        for letter in line:
            offset += 1
            if letter == ' ':
                yield offset

result = list(index_words_iter(address))
print(result[:3])
                
with open('address.txt', 'r') as f:
    it = index_file(f)
    results = islice(it, 0, 3)
    print(list(it))


[0]
[0, 5, 11, 15, 21, 26]


# イテレータ

- メモリを考慮して大量のデータを扱う可能性がある場合はイテレーター、ジェネレータを使用する

In [25]:
def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

visits = [15, 35, 80]
percentages= normalize(visits)
print(percentages)

class ReadVisits(object):
    def __init__(self, data_path):
        self.data_path = data_path
    
    def __iter__(self):
        with open(self.data_path) as f:
            for line in f:
                yield int(line)

def normilize_defensive(numbers):
    if iter(numbers) is iter(numbers):
        raise TypeError('Must supply a container')
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

print(normilize_defensive(visits))
visits = ReadVisits('./test.txt')
percentages= normalize(visits)
print(percentages)

print(normilize_defensive(visits))


[11.538461538461538, 26.923076923076923, 61.53846153846154]
[11.538461538461538, 26.923076923076923, 61.53846153846154]
[11.538461538461538, 26.923076923076923, 61.53846153846154]
[11.538461538461538, 26.923076923076923, 61.53846153846154]


# 可変長引数

- 見た目をスッキリさせる
- ジェネレータを渡すとタプルに変換するのでメモリを使い果す可能性があるため、明示的に少ない引数しか渡さない時しか使えない
- 引数の使い方に注意しないとバグを埋め込む可能性があるのでキーワード専用引数を使用する

In [26]:
def log(sequence, message, *values):
    if not values:
        print('%s: %s' % (sequence, message))
    else:
        values_str = ','.join(str(x) for x in values)
        print('%s: %s: %s' % (sequence, message, values_str))

log(1, 'Favorites', 7, 33)
log('Favorites', 7, 33)

1: Favorites: 7,33
Favorites: 7: 33


# デフォルト引数に関して

In [29]:
import json

def decode(data, default=None):
    """
    Args:
       data: Json data to decode
       default: Value to return if decoding fails.
           Defaults to an empty dictionary
    """
    if default is None:
        default = {}
    try:
        return json.loads(data)
    except ValueError:
        return default

foo = decode('bad data')
foo['stuff'] = 5
bar = decode('bad data')
bar['meep'] = 1

print(foo)
print(bar)

{'stuff': 5}
{'meep': 1}


# キーワード専用の引数

In [33]:
def safe_division(number, divisor, *, ignore_overflow=False, ignore_sero_division=False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_sero_division:
            return float('inf')
        else:
            raise
            
safe_division(1.0, 10**500, ignore_overflow=True, ignore_sero_division=False)

0